In [1]:
import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

c:\Users\Administrator\Desktop\FinetuningOptuna\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("heart.csv")
df = pd.get_dummies(df)
df.info()
X = df.drop("HeartDisease", axis=1).copy()
y = df["HeartDisease"].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=369)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                918 non-null    int64  
 1   RestingBP          918 non-null    int64  
 2   Cholesterol        918 non-null    int64  
 3   FastingBS          918 non-null    int64  
 4   MaxHR              918 non-null    int64  
 5   Oldpeak            918 non-null    float64
 6   HeartDisease       918 non-null    int64  
 7   Sex_F              918 non-null    bool   
 8   Sex_M              918 non-null    bool   
 9   ChestPainType_ASY  918 non-null    bool   
 10  ChestPainType_ATA  918 non-null    bool   
 11  ChestPainType_NAP  918 non-null    bool   
 12  ChestPainType_TA   918 non-null    bool   
 13  RestingECG_LVH     918 non-null    bool   
 14  RestingECG_Normal  918 non-null    bool   
 15  RestingECG_ST      918 non-null    bool   
 16  ExerciseAngina_N   918 non

In [3]:
model = SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

0.7427536231884058


In [4]:
def objective(trial):
    hyperparams = {
        "C": trial.suggest_float("C", 1e-3, 1e3, log=True),
        "kernel": trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"]),
        "gamma": trial.suggest_categorical("gamma", ["scale", "auto"]),
        "shrinking": trial.suggest_categorical("shrinking", [True, False]),
        "probability": trial.suggest_categorical("probability", [True, False]) ,
        "tol": trial.suggest_float("tol", 1e-5, 1e-2, log=True),
        "max_iter": -1,  
    }
    if hyperparams["kernel"] == "poly":
        hyperparams["degree"] = trial.suggest_int("degree", 2, 5)
        hyperparams["coef0"] = trial.suggest_float("coef0", -1.0, 1.0)   
    if hyperparams["kernel"] in ["poly", "sigmoid"]:
        hyperparams["coef0"] = trial.suggest_float("coef0", -1.0, 1.0)
    model = SVC(**hyperparams, random_state=369)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return acc

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10, n_jobs=-1)

[I 2025-08-21 21:08:55,549] A new study created in memory with name: no-name-49d9970f-0172-4449-97d2-8c6e2f8fde71
[I 2025-08-21 21:08:55,613] Trial 4 finished with value: 0.5434782608695652 and parameters: {'C': 2.3207523438543216, 'kernel': 'sigmoid', 'gamma': 'auto', 'shrinking': True, 'probability': False, 'tol': 0.0004948189852368622, 'coef0': -0.9603034571278048}. Best is trial 4 with value: 0.5434782608695652.
[I 2025-08-21 21:08:55,637] Trial 5 finished with value: 0.5434782608695652 and parameters: {'C': 0.02573340326499061, 'kernel': 'sigmoid', 'gamma': 'auto', 'shrinking': True, 'probability': False, 'tol': 0.0008308835065827481, 'coef0': 0.417882789627688}. Best is trial 4 with value: 0.5434782608695652.
[I 2025-08-21 21:08:55,648] Trial 6 finished with value: 0.5434782608695652 and parameters: {'C': 1.870257922916628, 'kernel': 'sigmoid', 'gamma': 'auto', 'shrinking': True, 'probability': False, 'tol': 9.347940812027059e-05, 'coef0': -0.4835612544634058}. Best is trial 4 wi

In [5]:
best_hiper = study.best_params
model = SVC(**best_hiper, random_state=369)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

0.8731884057971014
